In [1]:
from datasets import load_dataset

dataset = load_dataset(
    "json",
    data_files={"train": "../datasets/python_states_singleline.json"},
)

/Users/nikh/opt/miniconda3/envs/nlp_hw4/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 8968897 examples [00:12, 691533.98 examples/s]


In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")


def tokenize_function(examples):
    # I didn't add any padding. maybe we need to...
    model_inputs = tokenizer(examples["input"], truncation=True, max_length=512)
    labels = tokenizer(examples["output"], truncation=True, max_length=512).input_ids
    model_inputs["labels"] = labels
    return model_inputs


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 8968897/8968897 [04:29<00:00, 33290.59 examples/s]


In [5]:
tokenized_datasets.save_to_disk("../datasets")

Saving the dataset (12/12 shards): 100%|██████████| 8968897/8968897 [00:23<00:00, 385294.22 examples/s]


In [ ]:
from datasets import load_from_disk
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments

# tokenized_datasets = load_from_disk('/path/to/tokenized_datasets')

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1")

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    # Make sure to include tokenizer if needed, e.g., for decoding in evaluation
)